In [8]:
import pandas as pd
import os
import numpy as np

In [12]:
folder = "./data"

files = os.listdir(folder)

datasets = {file_name.replace(".xlsx", ""): pd.read_excel(os.path.join(folder, file_name)) for file_name in files}

for name, df in datasets.items():
    print(f"\n{name} loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    print(df.apply(lambda col: col.unique()))


EOAL_PAIS_DL loaded: 943 rows, 7 columns
Unnamed: 0          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
AÑO                                          [2022, 2023, 2024, 2025]
MES                           [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
PAIS                [Alemania, Austria, Bélgica, Dinamarca, España...
LUGAR_RESIDENCIA    [Residentes en el Extranjero, Residentes en Es...
VIAJEROS            [2892, 0, 717, 39580, 3798, 3592, 2898, 8086, ...
PERNOCTACIONES      [6133, 0, 1603, 86870, 8338, 8065, 6587, 17311...
dtype: object

CST_GASTO_CONSUMO_TURISTICO_DL loaded: 112 rows, 9 columns
Unnamed: 0                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
AÑO                                          [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
PRODUCTO                                    [Agencias de viajes y otros servicios de reser...
NUM_GASTO_INTERIOR                          [6910.6, 23393.6, 28999.0, 86710.1, 5383.0, 77...
NUM_GASTO_INTERNO 

In [ ]:
# ¿Están los turistas dispuestos a pagar más por opciones sostenibles y auténticas?

In [32]:
datasets["EGATUR_DL"].columns.tolist()

['AÑO',
 'MES',
 'PAIS_RESIDENCIA',
 'TIPO_VISITANTE',
 'CCAA_DESTINO',
 'PERNOCTACIONES',
 'GASTO_TOTAL']

In [28]:
excursionistas = datasets["EGATUR_DL"]["TIPO_VISITANTE"].isin(["Excursionista no residente (no tránsito)", "Excursionista no residente en tránsito"]).sum()

np.int64(4437)

In [33]:
evolucion = datasets["EGATUR_DL"].groupby(["AÑO", "TIPO_VISITANTE"]).size().reset_index(name="TOTAL")

In [36]:
egatur_dl = datasets["EGATUR_DL"].copy()

egatur_dl["TIPO_VISITANTE_GROUP"] = egatur_dl["TIPO_VISITANTE"].apply(lambda x: "Turista" if "Turista" in x else "Excursionista")

In [56]:
evolucion = egatur_dl.groupby(["AÑO", "TIPO_VISITANTE_GROUP"]).size().reset_index(name="TOTAL")
visitantes_ccaa = (egatur_dl.groupby(["AÑO", "CCAA_DESTINO", "TIPO_VISITANTE_GROUP"]).size().reset_index(name="TOTAL"))

In [53]:
evolucion

,AÑO,TIPO_VISITANTE_GROUP,TOTAL
0,2018,Excursionista,1077
1,2018,Turista,3613
2,2019,Excursionista,1140
3,2019,Turista,3612
4,2020,Excursionista,597
5,2020,Turista,2474
6,2021,Excursionista,730
7,2021,Turista,3209
8,2022,Excursionista,893
9,2022,Turista,3478


In [54]:
evolucion_total_anual

,AÑO,TOTAL
0,2018,4690
1,2019,4752
2,2020,3071
3,2021,3939
4,2022,4371
5,2023,3672
6,2024,3489
7,2025,1430


In [85]:
ccaa_tradicionales = ["Cataluña", "Andalucía", "Comunidad Valenciana", "Illes Balears", "Madrid", "Canarias"]
ccaa_alternativas = ["Galicia", "Cantabria", "Aragón", "Extremadura", "Castilla y León", "La Rioja", "Asturias", "Navarra", "País Vasco"]

def clasificar_ccaa(ccaa):
    if ccaa in ccaa_tradicionales:
        return "Tradicional"
    elif ccaa in ccaa_alternativas:
        return "Emergente"
    else:
        return "Otra"

visitantes_ccaa["TIPO_CCAA"] = visitantes_ccaa["CCAA_DESTINO"].apply(clasificar_ccaa)

visitantes_ccaa.to_csv("./exports/visitantes_ccaa.csv", index=False)
visitantes_ccaa.sort_values(by="TOTAL", ascending=False)

,AÑO,CCAA_DESTINO,TIPO_VISITANTE_GROUP,TOTAL,TIPO_CCAA
13,2018,Cataluña,Turista,325,Tradicional
51,2019,Cataluña,Turista,324,Tradicional
205,2024,Cataluña,Turista,323,Tradicional
186,2023,Cataluña,Turista,320,Tradicional
39,2019,Andalucía,Turista,318,Tradicional
...,...,...,...,...,...
29,2018,Melilla,Excursionista,1,Otra
156,2022,Castilla-La Mancha,Excursionista,1,Otra
141,2021,Principado de Asturias,Excursionista,1,Otra
119,2021,Castilla-La Mancha,Excursionista,1,Otra


In [ ]:
# Calcular el crecimiento de los tipo de turistas por cada tipo de CCAA(Emergente vs Tradicional)
# 1. Filtrar el tipo de visitantes y los años (???solo hasta 2022 porque a partir de ese año no existen "Excursionistas")

In [100]:
turistas = visitantes_ccaa[(visitantes_ccaa["TIPO_VISITANTE_GROUP"] == "Turista")]
turistas_resumen = (
    turistas.groupby(["AÑO", "TIPO_CCAA"])["TOTAL"]
    .sum()
    .reset_index()
)

turistas_pivot = turistas_resumen.pivot(
    index="AÑO",
    columns="TIPO_CCAA",
    values="TOTAL"
)

turistas_pivot

TIPO_CCAA,Emergente,Otra,Tradicional
AÑO,,,
2018,1275,1268,1070
2019,1317,1244,1051
2020,875,846,753
2021,1181,1094,934
2022,1242,1205,1031
2023,1363,1229,1080
2024,1251,1208,1030
2025,507,483,440


In [106]:
crecimiento_abs = turistas_pivot.loc[2024] - turistas_pivot.loc[2018]
crecimiento_porcentaje = ((crecimiento_abs / turistas_pivot.loc[2018]) * 100).round(2)

crecimiento_porcentaje

TIPO_CCAA
Emergente     -1.88
Otra          -4.73
Tradicional   -3.74
dtype: float64

In [125]:
visitantes_ccaa_pivot = visitantes_ccaa.groupby(["AÑO", "TIPO_VISITANTE_GROUP", "TIPO_CCAA"])["TOTAL"].sum().reset_index()
visitantes_ccaa_pivot

visitantes_ccaa_pivot.to_csv("./exports/visitantes_ccaa_pivot.csv", index=False)


In [61]:
totales_por_ccaa = visitantes_ccaa.groupby("CCAA_DESTINO")["TOTAL"].sum().reset_index()
totales_por_ccaa.sort_values(by="TOTAL", ascending=False)

,CCAA_DESTINO,TOTAL
6,Cataluña,2948
0,Andalucía,2700
9,Comunidad de Madrid,2427
10,Comunitat Valenciana,2311
16,País Vasco,2231
4,Castilla y León,1961
13,Illes Balears,1936
12,Galicia,1802
2,Canarias,1703
1,Aragón,1399


In [126]:
pivot_table = visitantes_ccaa_pivot.pivot_table(
    index="AÑO",
    columns=["TIPO_VISITANTE_GROUP", "TIPO_CCAA"],
    values="TOTAL"
)

pivot_table

TIPO_VISITANTE_GROUP Excursionista                      Turista          \
TIPO_CCAA                Emergente   Otra Tradicional Emergente    Otra   
AÑO                                                                       
2018                         230.0  349.0       498.0    1275.0  1268.0   
2019                         268.0  353.0       519.0    1317.0  1244.0   
2020                         157.0  226.0       214.0     875.0   846.0   
2021                         185.0  257.0       288.0    1181.0  1094.0   
2022                         216.0  298.0       379.0    1242.0  1205.0   
2023                           NaN    NaN         NaN    1363.0  1229.0   
2024                           NaN    NaN         NaN    1251.0  1208.0   
2025                           NaN    NaN         NaN     507.0   483.0   

TIPO_VISITANTE_GROUP              
TIPO_CCAA            Tradicional  
AÑO                               
2018                      1070.0  
2019                      1051.0  
2020                       753.0  
2021                       934.0  
2022                      1031.0  
2023                      1080.0  
2024                      1030.0  
2025                       440.0

In [119]:
valores = pivot_table.loc[2018:2022].values  # solo hasta 2022
columnas = pivot_table.columns

crecimiento_porcentaje = ((valores[-1] - valores[0]) / valores[0]) * 100

for i, col in enumerate(columnas):
    tipo, ccaa = col
    print(f"{tipo} en CCAA {ccaa}: {crecimiento_pct[i]:.2f}% de crecimiento entre 2018 y 2022")


Excursionista en CCAA Emergente: -6.09% de crecimiento entre 2018 y 2022
Excursionista en CCAA Otra: -14.61% de crecimiento entre 2018 y 2022
Excursionista en CCAA Tradicional: -23.90% de crecimiento entre 2018 y 2022
Turista en CCAA Emergente: -2.59% de crecimiento entre 2018 y 2022
Turista en CCAA Otra: -4.97% de crecimiento entre 2018 y 2022
Turista en CCAA Tradicional: -3.64% de crecimiento entre 2018 y 2022


MultiIndex([('Excursionista',   'Emergente'),
            ('Excursionista',        'Otra'),
            ('Excursionista', 'Tradicional'),
            (      'Turista',   'Emergente'),
            (      'Turista',        'Otra'),
            (      'Turista', 'Tradicional')],
           names=['TIPO_VISITANTE_GROUP', 'TIPO_CCAA'])

In [76]:
visitantes_ccaa_pivot.to_csv("./exports/visitantes_ccaa_pivot.csv", index=False)